In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import re
from sklearn.feature_extraction.text import CountVectorizer
from wilds.common.data_loaders import get_train_loader
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [2]:
def CleanText(text):
    text = re.sub(r'''[\[|\]]''', "", text).split()
    text = np.array(text, dtype="float64")
    return text

In [4]:
df = pd.read_csv("all_data_with_identitiesEmbedded.csv")

df = df.loc[:, ["comment_text", "split", "na_gender", "na_race", "toxicity", "male", "female", "transgender", "black", "white", "asian", "latino"]]

df['toxicity'] = df['toxicity'].apply(lambda score: np.round(score))
df['comment_text'] = df['comment_text'].apply(lambda text: CleanText(text))

df['male'] = df['male'].apply(lambda x: round(x))
df['female'] = df['female'].apply(lambda x: round(x))
df['transgender'] = df['transgender'].apply(lambda x: round(x))
df['black'] = df['black'].apply(lambda x: round(x))
df['white'] = df['white'].apply(lambda x: round(x))
df['asian'] = df['asian'].apply(lambda x: round(x))
df['latino'] = df['latino'].apply(lambda x: round(x))

In [5]:
# Updating values for training_data
training_data = df[df['split'] == 'train']
training_data = training_data[(training_data['na_gender'] == 0) | (training_data['na_race'] == 0)]

# Getting test_data
test_data = df[df['split'] == 'test']
test_data = test_data[(test_data['na_gender'] == 0) | (test_data['na_race'] == 0)]

# Getting validation_data
validation_data = df[df['split'] == 'val']
validation_data = validation_data[(validation_data['na_gender'] == 0) | (validation_data['na_race'] == 0)]

In [72]:
# Reshaping data for LSTM
X_train = training_data['comment_text'].values.tolist()
X_train = np.array(X_train).reshape(len(X_train), 24, 32)
Y_train = training_data['toxicity'].values.tolist()
Y_train = np.array(Y_train).reshape(len(Y_train), 1)

X_test = test_data['comment_text'].values.tolist()
X_test = np.array(X_test).reshape(len(X_test), 24, 32)
Y_test = test_data['toxicity'].values.tolist()
Y_test = np.array(Y_test).reshape(len(Y_test), 1)

X_val = validation_data['comment_text'].values.tolist()
X_val = np.array(X_val).reshape(len(X_val), 24, 32)
Y_val = validation_data['toxicity'].values.tolist()
Y_val = np.array(Y_val).reshape(len(Y_val), 1)

In [73]:
from tensorflow import keras
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers.core import Dense
from tensorflow.python.keras.layers.embeddings import Embedding
from tensorflow.python.keras.layers.cudnn_recurrent import CuDNNLSTM
from tensorflow.python.keras.layers.core  import Activation
from tensorflow.python.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.python.keras.layers.core import Dropout
from tensorflow.python.keras.layers import Bidirectional
from tensorflow.python.keras.layers import GlobalMaxPool1D, MaxPooling1D, Conv1D
from tensorflow.python.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import LSTM

In [78]:
model = keras.Sequential()
#data_dim = 300
#timesteps = 400
#e = Embedding(embed_matrix_test.shape[0], 300, weights=[embed_matrix_test], input_length=400, trainable=False)

samples, timesteps, features = X_train.shape

print((X_train.shape[1], 1))
regressor = Sequential()

regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 32)))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

regressor.add(Dense(units = 1))

regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

regressor.fit(X_train, Y_train, epochs = 100, batch_size = 32)




#model = Model(inputs=[input_text], outputs=pred)
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#x1 = layers.Dense(64, activation="relu")(inputs)



#model.add(e)
#model.add(Dropout(0.2))
#model.add(Conv1D(filters=300, kernel_size=4, activation='relu'))
#model.add(MaxPooling1D(pool_size=4))
#model.add(Bidirectional(CuDNNLSTM(300, return_sequences=True,
#               input_shape=(100, timesteps, data_dim),  stateful=False)))
#model.add(CuDNNLSTM(128, stateful= False,))
#model.add(GlobalMaxPool1D())
#model.add(Dropout(0.2))
#model.add(Dense(300, activation='relu'))
#model.add(Dropout(0.2))
#model.add(Dense(1, activation='sigmoid', input_shape=(300,)))

(24, 1)
Epoch 1/100


UnknownError:    Fail to find the dnn implementation.
	 [[{{node CudnnRNN}}]]
	 [[sequential_31/lstm_25/PartitionedCall]] [Op:__inference_train_function_467993]

Function call stack:
train_function -> train_function -> train_function
